In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver
from method.ours.parse import (
    initial_parse,
    parse_after_feedback,
)

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

In [4]:
driver = create_driver(HEADLESS)
driver.get('https://www.aircanada.com/ca/en/aco/home.html')

In [5]:
# form = driver.find_elements(By.TAG_NAME, 'form')[48]
# form = driver.find_element(By.ID, 'register')
form = driver.find_elements(By.TAG_NAME, 'form')[1]

# Processing

In [7]:
relation_graph, input_groups = initial_parse(driver, form, TEXT_EMBEDDING_METHOD)

Computing transition probabilities:   0%|          | 0/340 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:09<00:00,  3.61it/s]


In [8]:
relation_graph.edges()

[edge FOR weight 1 from <label> Round-trip </label> at y: (572, 592), x: (249, 319) to <input>value is R</input> at y: (567, 597), x: (219, 319),
 edge FOR weight 1 from <input>value is R</input> at y: (567, 597), x: (219, 319) to <label> Round-trip </label> at y: (572, 592), x: (249, 319),
 edge FOR weight 1 from <label> One-way </label> at y: (572, 592), x: (364, 421) to <input>value is O</input> at y: (567, 597), x: (334, 421),
 edge FOR weight 1 from <input>value is O</input> at y: (567, 597), x: (334, 421) to <label> One-way </label> at y: (572, 592), x: (364, 421),
 edge FOR weight 1 from <label> Multi-city/Stopover </label> at y: (572, 592), x: (467, 591) to <input>value is M</input> at y: (567, 597), x: (437, 591),
 edge FOR weight 1 from <input>value is M</input> at y: (567, 597), x: (437, 591) to <label> Multi-city/Stopover </label> at y: (572, 592), x: (467, 591),
 edge FOR weight 1 from <label> Book with points </label> at y: (574, 594), x: (682, 905) to <input></input> at 

# Generation

In [9]:
from method.ours.constraints import generate_constraints_from_string
from method.ours.prompts import (
    get_form_context,
    create_constraint_generation_user_prompt,
    create_value_generation_user_prompt,
)
from method.ours.generation import (
    generate_constraints_with_llm,
    generate_values_with_llm,
)

In [10]:
form_context = get_form_context(input_groups)

In [11]:
class FillTable:
    def __init__(self):
        self.entries = {}
    
    
    def add_entry(self, field_id, input_group, constraints):
        self.entries[field_id] = FillTableRow(input_group, constraints)
    
    
    def get_entry_by_field_id(self, field_id):
        return self.entries[field_id]
    
    
    def get_entry_by_input_group(self, input_group):
        filtered = list(filter(
            lambda x: x.input_group.node.xpath == input_group.node.xpath,
            self.entries.values()
        ))
        if len(filtered) > 0:
            return filtered[0]
        return None


class FillTableRow:
    def __init__(self, input_group, constraints):
        self.input_group = input_group
        self.constraints = constraints
        self.feedback = None
        self.value = None
    
    
    def set_value(self, value):
        self.value = value
    
    
    def set_feedback(self, feedback):
        self.feedback = feedback

In [12]:
fill_table = FillTable()

In [13]:
for input_group in input_groups:
    # TODO: set more conditions on <button> to be sure that buttons here are inputs
    if input_group.node.element.name == 'button':
        continue
    
    constraint_user_prompt = create_constraint_generation_user_prompt(form_context, input_group)
    
    generated_constraints = generate_constraints_with_llm(
        constraint_user_prompt,
        openai_api_key=openai.api_key
    )
    
    field_name, constraints = generate_constraints_from_string(generated_constraints)
    
    fill_table.add_entry(field_name, input_group, constraints)

  Prompt: [
  {
    "role": "system",
    "content": "\nYour task is to generate a set of constraints for form fields. Your decisions must be made independently without seeking user assistance or additional information.\n\nThe list of constraints that you must choose from, and their signatures, is as follows:\n\n1. toBe(value) # the input field value exactly matches the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondition(condition, value) # the input field value has the given condition to the given value\n5. toContainSubstr(value) # the input field value contains a specific string\n6. toContainChar(value) # the input field value contains a specific character\n7. toBeAlpha() # the input field value is alphabetic\n8. toBeNumeric() # the input field value is numeric\n9. toBeAlphaNumeric

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('tripType'))\n.toBe('M')",
        "role": "assistant"
      }
    }
  ],
  "created": 1688147691,
  "id": "chatcmpl-7XCcV1HnrDvXkygU4qi8NUM5VS6wk",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 818,
    "total_tokens": 828
  }
}
  Total running cost: $0.075
  Prompt: [
  {
    "role": "system",
    "content": "\nYour task is to generate a set of constraints for form fields. Your decisions must be made independently without seeking user assistance or additional information.\n\nThe list of constraints that you must choose from, and their signatures, is as follows:\n\n1. toBe(value) # the input field value exactly matches the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field va

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('To'))\n.toBeAlpha()\n.toHaveLengthCondition('>', 2)\n.toHaveWhiteSpace()\n.not.toBe(field('From'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1688147698,
  "id": "chatcmpl-7XCcc04J4aAt5jrhGMA7VLXYa9ui8",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 797,
    "total_tokens": 824
  }
}
  Total running cost: $0.147
  Prompt: [
  {
    "role": "system",
    "content": "\nYour task is to generate a set of constraints for form fields. Your decisions must be made independently without seeking user assistance or additional information.\n\nThe list of constraints that you must choose from, and their signatures, is as follows:\n\n1. toBe(value) # the input field value exactly matches the given value\n2. toHaveLengthCondition(condition, value) # the length of the input fi

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('Return'))\n.toBeNumeric()\n.toHaveLengthCondition('=', 4)\n.toContainChar('/')\n.toStartWith(field('Departure'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1688147707,
  "id": "chatcmpl-7XCcloR9MeBE61h8fDKBAujemgQrM",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 29,
    "prompt_tokens": 849,
    "total_tokens": 878
  }
}
  Total running cost: $0.201


In [14]:
def split_constant_and_field_constraints(constraints):
    constants = list(filter(lambda x: not x.is_field, constraints))
    fields = list(filter(lambda x: x.is_field, constraints))
    return constants, fields

In [15]:
# value generation loop

for input_group in input_groups:
    if input_group.node.element.name == 'button':
        continue
    
    fill_entry = fill_table.get_entry_by_input_group(input_group)
    
    if fill_entry is None:
        continue
    
    constraints = fill_entry.constraints
    constant_constraints, field_constraints = split_constant_and_field_constraints(constraints)
    including_constraints = [*constant_constraints]

    relevant_field_values = []
    for field_constraint in field_constraints:
        field_arg = field_constraint.get_field_args()[0]
        
        if field_arg not in fill_table.entries:
            continue
        
        value = fill_table.get_entry_by_field_id(field_arg).value
        
        if value is not None:
            including_constraints.append(field_constraint)
            relevant_field_values.append((field_arg, value))
    
    value_user_prompt = create_value_generation_user_prompt(
        fill_entry.input_group,
        including_constraints,
        relevant_field_values=relevant_field_values if len(relevant_field_values) > 0 else None,
    )
    
    generated_value = generate_values_with_llm(
        value_user_prompt,
        openai_api_key=openai.api_key
    )
    
    fill_entry.set_value(generated_value)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a form field based on the form field information and a set of constraints on the field. Your decisions must always be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfy the constraints while keeping in mind the nature of the input from the available information.\nOnly generate the value and don't explain your answers.\nOnly generate value for the inputs in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_tripTypeSelector_M\" name=\"tripType\" type=\"radio\" value=\"M\"/>\nwith label: Multi-city/Stopover\n    The relevant information available in the form are (in order of relevance):\n    1.\ninput: <input id=\"bkmgFlights_tripTypeSelector_R\" nam

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "20/12",
        "role": "assistant"
      }
    }
  ],
  "created": 1688147758,
  "id": "chatcmpl-7XCdaZdKX96jIe7VloYMvKHIEyBB0",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 3,
    "prompt_tokens": 327,
    "total_tokens": 330
  }
}
  Total running cost: $0.245
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a form field based on the form field information and a set of constraints on the field. Your decisions must always be made independently without seeking user assistance or additional information.\nFor each user prompt, you need to generate one value that satisfy the constraints while keeping in mind the nature of the input from the available information.\nOnly generate the value and don't explain your answers.\nOnly generate value for the inputs in question, and not the

In [16]:
from method.ours.utils import interact_with_input

In [17]:
for input_group in input_groups:
    entry = fill_table.get_entry_by_input_group(input_group)
    
    if entry is None:
        continue
    
    print(entry.input_group.node)
    print(entry.value)
    
    element = driver.find_element(By.XPATH, entry.input_group.node.xpath)
    
    if element.get_attribute('type') not in ['submit', 'radio', 'checkbox']:
        interact_with_input(element, entry.value)

<input>value is M</input> at y: (567, 597), x: (437, 591)
M
<input></input> at y: (574, 594), x: (652, 905)
true
<input></input> at y: (628, 678), x: (198, 466)
New York
<input></input> at y: (628, 678), x: (536, 804)
Los Angeles
<input>DD/MM</input> at y: (628, 678), x: (833, 958)
20/12
<input>DD/MM</input> at y: (628, 678), x: (978, 1093)
25/12


In [18]:
submit = list(filter(
    lambda x: x.node.element.attrs['type'] == 'submit',
    input_groups
))[0]

In [19]:
interact_with_input(driver.find_element(By.XPATH, submit.node.xpath), True)

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x00000001100a76b8 chromedriver + 4937400
1   chromedriver                        0x000000011009eb73 chromedriver + 4901747
2   chromedriver                        0x000000010fc5c616 chromedriver + 435734
3   chromedriver                        0x000000010fc42bc1 chromedriver + 330689
4   chromedriver                        0x000000010fc42923 chromedriver + 330019
5   chromedriver                        0x000000010fc5e262 chromedriver + 442978
6   chromedriver                        0x000000010fcde56e chromedriver + 968046
7   chromedriver                        0x000000010fcc3de3 chromedriver + 859619
8   chromedriver                        0x000000010fc91d7f chromedriver + 654719
9   chromedriver                        0x000000010fc930de chromed

# Feedback with Diff

In [20]:
html1 = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.198)
Stacktrace:
0   chromedriver                        0x00000001100a76b8 chromedriver + 4937400
1   chromedriver                        0x000000011009eb73 chromedriver + 4901747
2   chromedriver                        0x000000010fc5c616 chromedriver + 435734
3   chromedriver                        0x000000010fc42bc1 chromedriver + 330689
4   chromedriver                        0x000000010fc42923 chromedriver + 330019
5   chromedriver                        0x000000010fc5e262 chromedriver + 442978
6   chromedriver                        0x000000010fcde56e chromedriver + 968046
7   chromedriver                        0x000000010fcc3de3 chromedriver + 859619
8   chromedriver                        0x000000010fc91d7f chromedriver + 654719
9   chromedriver                        0x000000010fc930de chromed

In [8]:
html2 = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')

In [9]:
from bs4 import BeautifulSoup
from deepdiff import DeepDiff

# Parse the HTML.
soup1 = BeautifulSoup(html1, 'html.parser')
soup2 = BeautifulSoup(html2, 'html.parser')

# Find the differences.
diff = DeepDiff(soup1, soup2)

# Print the differences.
for change, details in diff.items():
    print(change)
    for detail in details:
        print("   ", detail)

type_changes
    root[0][8][0][2][0][1][1][4][0][0][0][0][3][0][3][0][2]
    root[0][8][0][2][0][1][1][4][0][0][0][0][3][0][3][1][0][1][3][0][3][0][1][0][0][0][0][1][1][0]
    root[0][8][0][2][0][1][1][4][0][0][0][0][3][0][3][1][0][1][3][0][3][0][1][0][0][0][0][1][1][2]
    root[0][8][0][2][0][1][1][4][0][0][0][0][3][0][3][1][0][1][3][0][3][0][1][0][0][0][0][1][1][3]
    root[0][8][0][2][0][1][1][4][0][0][0][0][3][0][3][1][0][1][3][0][4][1][0][0][0][0][0][0][1][1][0]
    root[0][8][0][2][0][1][1][4][0][0][0][0][3][0][3][1][0][1][3][0][4][1][0][0][0][0][0][0][1][1][2]
    root[0][8][0][2][0][1][1][4][0][0][0][0][3][0][3][1][0][1][3][0][4][1][0][0][0][0][0][0][1][1][3]
    root[0][8][0][2][0][1][1][4][0][0][0][0][3][0][3][1][0][1][3][0][4][1][0][1][0][0][0][0][1][1][0]
    root[0][8][0][2][0][1][1][4][0][0][0][0][3][0][3][1][0][1][3][0][4][1][0][1][0][0][0][0][1][1][2]
    root[0][8][0][2][0][1][1][4][0][0][0][0][3][0][3][1][0][1][3][0][4][1][0][1][0][0][0][0][1][1][3]
values_changed
   

In [26]:
feedback_keywords = [
    'invalid',
    'required',
    'error',
    'not possible',
    'missing',
    'failed',
    'incorrect',
    'retry',
    'verify',
    'exceeds',
    'denied',
    'please enter',
    'format',
    'does not match',
    'try again',
    'unavailable',
    'warning',
    'out of range',
    'unable to',
    'not recognized',
    'not allowed',
    'incomplete',
    'cannot process',
    'duplicate',
    'too short',
    'too long',
    'doesn\'t exist',
    'does not exist',
    'already exist',
    'authentication failed',
    'expired',
    'update failed',
    'connection',
    'server error',
    'system error',
    'not valid',
    'timed out',
    'access denied',
    'please confirm',
    'not acceptable',
    'submission failed',
    'unexpected',
    'conflict',
    'not applicable',
    'not active',
    'not found',
    'not secure',
    'forbidden',
    'disconnect',
    'prohibited',
]

In [28]:
changes = []

for type_change in diff['type_changes'].values():
    changes.append(type_change['new_value'])
for value_change in diff['values_changed'].values():
    changes.append(value_change['new_value'])
for iterable_item in diff['iterable_item_added'].values():
    changes.append(iterable_item)

In [34]:
changes = list(filter(lambda x: x.strip() != '', map(lambda x: x.text.strip(), changes)))

In [35]:
def has_feedback_keyword(sentence):
    for keyword in feedback_keywords:
        if keyword in sentence:
            return True
    return False


changes = list(filter(has_feedback_keyword, changes))

In [36]:
changes

['It is not possible to search for flights which have both an origin and a destination in the United States.',
 'It is not possible to book a flight segment between two cities in the same country, unless that country is Canada.  Please contact Air Canada ReservationsOpens in a new tab for assistance with this type of booking.']

# Feedback

In [25]:
new_form = driver.find_elements(By.TAG_NAME, 'form')[0]

In [27]:
new_relation_graph, new_input_groups = parse_after_feedback(driver, new_form, relation_graph, TEXT_EMBEDDING_METHOD)

Computing transition probabilities:   0%|          | 0/366 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:11<00:00,  3.39it/s]


In [28]:
relation_graph.diff(new_relation_graph)

{'added': [<span>New York </span> at y: (836, 858), x: (376, 451),
  <span>JFK</span> at y: (836, 858), x: (452, 474),
  <span>Fri Sep 15</span> at y: (836, 858), x: (658, 731),
  <span>Wed Sep 20</span> at y: (836, 858), x: (792, 880),
  <span-wrap> Enter the date day and month in this format: DD/MM, or use the 'Show Calendar' button to open the calendar and select your date from there. </span-wrap> at y: (865, 866), x: (657, 658)],
 'removed': [<span-wrap> Enter the date day and month in this format: DD/MM, or use the 'Show Calendar' button to open the calendar and select your date from there. </span-wrap> at y: (679, 680), x: (832, 833)]}

In [47]:
new_relation_graph.edges()

[edge FOR weight 1 from <label> Round-trip </label> at y: (748, 768), x: (249, 319) to <input>value is R</input> at y: (743, 773), x: (219, 319),
 edge FOR weight 1 from <input>value is R</input> at y: (743, 773), x: (219, 319) to <label> Round-trip </label> at y: (748, 768), x: (249, 319),
 edge FOR weight 1 from <label> One-way </label> at y: (748, 768), x: (364, 421) to <input>value is O</input> at y: (743, 773), x: (334, 421),
 edge FOR weight 1 from <input>value is O</input> at y: (743, 773), x: (334, 421) to <label> One-way </label> at y: (748, 768), x: (364, 421),
 edge FOR weight 1 from <label> Multi-city/Stopover </label> at y: (748, 768), x: (467, 591) to <input>value is M</input> at y: (743, 773), x: (437, 591),
 edge FOR weight 1 from <input>value is M</input> at y: (743, 773), x: (437, 591) to <label> Multi-city/Stopover </label> at y: (748, 768), x: (467, 591),
 edge FOR weight 1 from <label> Book with points </label> at y: (750, 770), x: (682, 905) to <input></input> at 